In [10]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(4, 4))
net(X)

tensor([[ 0.0256],
        [-0.1540],
        [-0.0315],
        [-0.0065]], grad_fn=<AddmmBackward0>)

### 参数的访问


In [67]:
print(net[-1].state_dict())


OrderedDict([('weight', tensor([[-0.2551,  0.0968, -0.0961,  0.2926,  0.1628,  0.2170, -0.1718,  0.0540]])), ('bias', tensor([-0.2362]))])


### 目标参数

In [42]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0988], requires_grad=True)
tensor([0.0988])


In [43]:
net[2].weight.grad == None

True

### 一次性访问所有参数

In [47]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


### 另一种访问网络参数的方式

In [78]:
net.state_dict()['0.weight'].data

tensor([[ 0.3638,  0.0163,  0.3752,  0.0682],
        [ 0.3603, -0.2424,  0.2326, -0.0673],
        [ 0.0213, -0.0546,  0.3088,  0.3284],
        [ 0.4775, -0.3056,  0.0288,  0.3061],
        [-0.2769, -0.4399,  0.3534, -0.3090],
        [-0.4094, -0.2218, -0.1083,  0.2048],
        [ 0.1376, -0.3175, -0.0053, -0.1794],
        [ 0.2755,  0.0765,  0.2305,  0.3555]])

### 从嵌套块收集参数

In [115]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[-0.4843],
        [-0.4845],
        [-0.4844],
        [-0.4844]], grad_fn=<AddmmBackward0>)

## 设计了网络后，我们看看它是如何工作的。


In [116]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [126]:
# 因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。
rgnet[0][1][0].bias.data

tensor([-0.4598,  0.3224,  0.3594,  0.0725, -0.0530,  0.1089,  0.2545,  0.3611])

## 参数初始化
### 内置初始化

让我们首先调用内置的初始化器。 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。

In [127]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 7.2825e-05, -2.3680e-04,  4.4941e-03, -6.2984e-03]), tensor(0.))

### 我们还可以将所有参数初始化为给定的常数，比如初始化为1。

In [7]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

### 我们还可以对某些块应用不同的初始化方法。 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

In [13]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.6187, -0.6256,  0.1356,  0.6567])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


### 自定义初始化

In [21]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init',*[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000,  0.0000,  6.3677],
        [-6.7339,  0.0000,  8.9807,  8.1806]], grad_fn=<SliceBackward0>)

### 注意，我们始终可以直接设置参数

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[1] = 42
net[0].weight.data[:]

tensor([[44.0000, 44.0000, 44.0000, 44.0000],
        [42.0000, 42.0000, 42.0000, 42.0000],
        [ 5.0000,  5.0000,  5.0000, -3.0080],
        [ 5.0000, -0.1148,  5.0000, 10.4332],
        [14.7347, 12.2062,  5.0000, 11.0128],
        [ 5.0000, -0.3276,  5.0000,  5.0000],
        [ 5.0000,  5.0000, 14.9888,  5.0000],
        [ 5.0000,  5.0000, 14.9611, 11.0141]])

## 参数绑定
有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [32]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保他们是一对象，而不是相同值
print(net[2].weight.data[0],
      net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([ 1.0000e+02, -1.4070e-01, -2.5368e-01,  3.5048e-02, -2.4149e-02,
        -2.7130e-01,  1.9860e-01,  1.3410e-01]) tensor([ 1.0000e+02, -1.4070e-01, -2.5368e-01,  3.5048e-02, -2.4149e-02,
        -2.7130e-01,  1.9860e-01,  1.3410e-01])
